In [1]:
#! git fetch origin
#! git reset --hard origin/master
from numba import cuda
device = cuda.get_current_device()
print(device)
device.reset()

%load_ext autoreload
%autoreload 2
get_base_data = False
from google.colab import drive
drive.mount('/content/drive')
drivef = "/content/drive/MyDrive/"
try:
  import open3d
except:
  ! pip install -q open3d
  import open3d
try:
  import open3d
except:
  ! pip install -q open3d
  import open3d
try:
  import ninja
except:
  ! pip install -q ninja
  import ninja
try:
  import h5py
except:
  ! pip install -q h5py
  import h5py
try:
  import transforms3d
except:
  ! pip install -q transforms3d
  import transforms3d

!mkdir /content/repo_folder
%cd /content/repo_folder
!git clone -q https://github.com/iwillnotkillyou/MSN-Point-Cloud-Completion.git
%cd ./MSN-Point-Cloud-Completion
!git checkout Pavel
#!git reset --hard
#!git pull

<CUDA device 0 'b'Tesla T4''>
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/repo_folder’: File exists
/content/repo_folder
fatal: destination path 'MSN-Point-Cloud-Completion' already exists and is not an empty directory.
/content/repo_folder/MSN-Point-Cloud-Completion
M	changes/dataset.py
M	changes/train.py
Already on 'Pavel'
Your branch is up to date with 'origin/Pavel'.


In [2]:
import os
dataf = "/content/repo_folder/MSN-Point-Cloud-Completion/data"
if get_base_data:
  if not os.path.isdir(f'{dataf}/val'):
    !unzip -n /content/drive/MyDrive/data/val.zip -d $dataf
  if not os.path.isdir(f'{dataf}/complete'):
    !unzip -n /content/drive/MyDrive/data/complete.zip -d $dataf
drivesplitf = "/content/drive/MyDrive/saved"
valp = f'{drivesplitf}/all.list'
valop = f'{drivef}/data/val.list'
!cp $valop $valp
%cd /content/repo_folder/MSN-Point-Cloud-Completion
!mkdir "/content/repo_folder/MSN-Point-Cloud-Completion/trained_model"
!cp "/content/drive/MyDrive/data/network.pth" "/content/repo_folder/MSN-Point-Cloud-Completion/trained_model/network.pth"

/content/repo_folder/MSN-Point-Cloud-Completion
mkdir: cannot create directory ‘/content/repo_folder/MSN-Point-Cloud-Completion/trained_model’: File exists


In [3]:
#! mkdir /content/saved
#! wget -nc -O /content/saved/network.pth https://www.dropbox.com/scl/fi/qx16y8hkfaq0b7nh1k4b2/network.pth?rlkey=w6yjdwy3zpbf25h2qtak6kkq5&dl=0

In [4]:
%cd /content/repo_folder
! git clone -q https://github.com/lynetcha/completion3d.git

import sys
sys.path.insert(1, '/content/repo_folder/MSN-Point-Cloud-Completion/')
sys.path.insert(1, '/content/repo_folder/MSN-Point-Cloud-Completion/changes')
sys.path.insert(1, '/content/repo_folder/MSN-Point-Cloud-Completion/emd')
sys.path.insert(1, '/content/repo_folder/completion3d/shared')
sys.path.insert(1, '/content/repo_folder/completion3d/shared/datasets')
sys.path.insert(1, '/content/repo_folder/completion3d/pytorch')

%cd /content/repo_folder/MSN-Point-Cloud-Completion/
from full_model import *
from transfered_model import *

/content/repo_folder
fatal: destination path 'completion3d' already exists and is not an empty directory.
/content/repo_folder/MSN-Point-Cloud-Completion


In [5]:
%load_ext autoreload
%autoreload 2
try:
  import trimesh
except:
  ! pip install -q trimesh
  import trimesh
%cd /content/repo_folder/MSN-Point-Cloud-Completion/
from model import *
from train import *
from my_chamfer_interface import chamferDist
class completionargs:
  def __init__(self,model, batch_size, name, fol):
    self.model = model
    self.name = name
    self.fol = fol
    self.batch_size = batch_size
    self.nworkers = 2
    self.dataset = 'shapenet'
    self.pc_augm_scale=0
    self.pc_augm_rot=0
    self.pc_augm_mirror_prob=0
    self.pc_augm_jitter=0
    self.inpts=500
  def step(self, targets, clouds_data, i):
    clouds_data = torch.from_numpy(clouds_data[1]).cuda()
    targets = torch.from_numpy(targets).cuda()
    output1, output2, emd1, emd2, expansion_penalty = self.model(clouds_data, targets.contiguous(), 0.005, 10)
    dist1, dist2 = chamferDist()(output2.float(),targets.float())
    loss = torch.mean(dist2) + torch.mean(dist1)
    if self.fol is not None:
      printf(clouds_data,output2,targets,i,f"{self.name}{int(torch.mean(emd1).item()*10000)}-{int(torch.mean(emd2).item()*10000)}-{int(torch.mean(loss).item()*10000)}",self.fol)
    return loss.item(), dist1, dist2, emd2, output2.detach().cpu().numpy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/content/repo_folder/MSN-Point-Cloud-Completion


In [11]:
%cd /content/repo_folder/MSN-Point-Cloud-Completion/
class PointNetfeatFreeze(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)

        self.bn1 = torch.nn.BatchNorm1d(64)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.bn3 = torch.nn.BatchNorm1d(1024)

    def freeze(self):
        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x, _ = torch.max(x, 2)
        x = x.view(-1, 1024)
        return x


import torch
from changes.train import *
from test_on_completion3d import test_on_completion3D
from matplotlib import pyplot as plt
import numpy as np

def make_fuller_model(name, args, architect_args):
  def additionalencoderf():
    if architect_args.additional_sizes is None:
      return AdditionalEncoderIndentity()
    return AdditionalEncoder(architect_args.additional_sizes,architect_args.additional_latents)
  def pointnetfeatf():
    if architect_args.modif_sizes is None:
      return PointNetfeatFreeze()
    if architect_args.modif_partial:
      return PointNetfeatReturn2TPartial(architect_args.modif_sizes)
    return PointNetfeatReturn2(architect_args.modif_sizes,architect_args.modif_latents)
  def additionaldecoderf():
    return GlobalTransformDepthSep(3, architect_args.additional_dec_sizes, 3, 1024)
  additionaldecoderf = None if architect_args.additional_dec_sizes is None else additionaldecoderf
  model = TransformMSN(additionalencoderf, pointnetfeatf, additionaldecoderf, args.num_points, architect_args.train_encoder)

  network = FullModel(model)
  model.cuda()
  #network = torch.nn.DataParallel(network)
  #model.apply(weights_init) #initialization of the weight
  if name != '':
      previous_state_dict = torch.load(name)
      model.load_state_dict(previous_state_dict,False,True)
      print("Previous weight loaded ")
  elif args.base_model != '':
      previous_state_dict = torch.load(args.base_model)
      model.load_state_dict(previous_state_dict,False,True)
      print("Base weight loaded ")
  model.freeze()
  network.cuda()
  return network

def make_base_model(args):
  model = MSN()
  network = FullModel(model)
  if args.base_model != '':
    previous_state_dict = torch.load(args.base_model)
    model.load_state_dict(previous_state_dict,False,True)
    print("Base weight loaded ")
  return network

train = True
class opt:
  def __init__(self):
    self.base_model = '/content/repo_folder/MSN-Point-Cloud-Completion/trained_model/network.pth'
    self.epoch_iter_limit_val = 10
    self.batchSize = 12
    self.epoch_iter_limit = None
    self.perc_data = 0.95
    self.perc_val_data = 0.1
    self.workers = 1
    self.nepoch = 10
    self.model = ''
    self.num_points = 8192
    self.n_primitives = 16
    self.env = "MSN_TRAIN"
    self.manualSeed = 0
    self.run_embedder = True
    self.embeddingpath = "/content/embeddings"
    self.usefirstorder = True

class architect_opts:
  def __init__(self, additional_enc = None, modif = None, additional_dec = None, shared_latents = None, modif_partial = True, train_encoder = False, train = train):
    dontuseshared = shared_latents is None
    def getval(tup):
      return tup[1] if dontuseshared and tup is not None else tup, tup[0] if dontuseshared and tup is not None else shared_latents

    self.additional_sizes, self.additional_latents = getval(additional_enc)
    self.modif_sizes, self.modif_latents = getval(modif)
    self.additional_dec_sizes, self.additional_dec_latents = getval(additional_dec)
    self.train = train
    self.modif_partial = modif_partial
    self.train_encoder = train_encoder

arcargsd = {}

argsd = {}
arcargsd = {}
arcargsd["m2"] = architect_opts((32,(256,256)),None,None,train = False,train_encoder=True)
arcargsd["m3"] = architect_opts((32,(128,128)),(32,(128,)),(32,(128,)),train = False)
arcargsd["m4"] = architect_opts((32,(256,256)),None,None,train = False,train_encoder=True)

torch.cuda.empty_cache()
names = arcargsd.keys()

for name in names:
  argsd[name] = opt()
argsd["m4"].model = "/content/drive/MyDrive/saved/m2/network.pth"
def makeso(args):
  args.usefirstorder = False
  args.batchSize = 64
drivesplitf = "/content/drive/MyDrive/saved"
#makeso(argsd["m2"])
#makeso(argsd["m4"])
#makeso(argsd["m6"])
resplit = True
trainon3df = True
if not trainon3df:
  trainp = f'{drivesplitf}/train.list'
  valp = f'{drivesplitf}/val.list'
  testp = f'{drivesplitf}/test.list'
  if resplit:
      make_data_splits(opt(),f'{drivesplitf}/all.list',
                      trainp,
                      valp,
                      testp)
else:
  !unzip -n /content/drive/MyDrive/data/3Dfuture.zip -d "/content/repo_folder/MSN-Point-Cloud-Completion/data"
  trainp = f'{drivesplitf}/train3dfuture.list'
  valp = f'{drivesplitf}/val3dfuture.list'
  testp = f'{drivesplitf}/test3dfuture.list'
  if resplit:
      make_data_splits(opt(),f'/content/drive/MyDrive/data/list3dfuture',
                      trainp,
                      valp,
                      testp)
def plotrainval(save_path):
  p = f"{save_path}/curves.npy"
  if not os.path.isfile(p):
    return
  curves = np.load(p)[1:]
  plt.plot(curves[0,:],label = "train emd")
  plt.plot(curves[1,:],label = "val emd")
  plt.legend()
  plt.show()
  plt.plot(curves[2,:],label = "train cd")
  plt.plot(curves[3,:],label = "val cd")
  plt.legend()
  plt.show()

if train:
  for name in names:
    if arcargsd[name].train:
      save_path = f"/content/drive/MyDrive/saved/{name}"
      model = make_fuller_model(argsd[name].model, argsd[name], arcargsd[name])
      cs = 0
      for c in model.model.children():
        csc = 0
        for param in c.parameters():
          if param.requires_grad:
            csc += param.nelement()
        cs += csc
        print(type(c), csc, cs)
      trainFull(model,save_path, argsd[name], 50, 1e-3, trainp=trainp,
              valp=valp,obj = True)
      plotrainval(save_path)


base_model = make_base_model(opt())
def print_results_sorted(results):
  vs = [(x,results[x]) for x in results]
  print([vs[x] for x in np.argsort([x[1] for x in vs])])


resultsavef = f'{drivesplitf}/output'
if False:
  dir = "/content/repo_folder/completion3d/data"
  link = "http://download.cs.stanford.edu/downloads/completion3d/dataset2019.zip"
  dataf = "/content/data.zip"
  ! wget -nc -O $dataf $link
  if not os.path.isdir(dir):
    ! mkdir -p $dir
    ! unzip -qq -o $dataf -d $dir
  %cd /content/repo_folder/completion3d/
  def testfunc(x, name):
    return test_on_completion3D(completionargs(x,4,name,None if False else resultsavef),False)
elif False:
  dataset_test = ShapeNet(testp, npoints=opt().num_points)
  dataset_test, _ = torch.utils.data.random_split(dataset_test,[0.1,0.9])
  dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size= 64,shuffle=False, num_workers=opt().workers, drop_last=True)
  def testfunc(x, name):
    return np.mean(test(x,dataloader_test,name,64,resultsavef+"based")[0])
else:
  !unzip -n /content/drive/MyDrive/data/3Dfuture.zip -d "/content/repo_folder/MSN-Point-Cloud-Completion/data"
  dataset_test = ShapeNetOBJ(testp)
  dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size= 64,shuffle=False, num_workers=opt().workers, drop_last=True)
  def testfunc(x, name):
    return np.mean(test(x,dataloader_test,name,64,resultsavef+"3dfuture")[0])
print(len(dataset_test))
results = {}
if True:
  model = make_base_model(opt())
  model.cuda()
  results["base_model"] = testfunc(model,"base_model")
  print_results_sorted(results)
  del(model)

for name in list(names):
  print(name)
  save_path = f"/content/drive/MyDrive/saved/{name}"
  save_file = f'{save_path}/network.pth'
  print(os.path.isfile(save_file))
  if not os.path.isfile(save_file):
    continue
  #plotrainval(save_path)
  model = make_fuller_model(save_file, argsd[name], arcargsd[name])
  model.cuda()
  results[name] = testfunc(model,name)
  print_results_sorted(results)
  del(model)

/content/repo_folder/MSN-Point-Cloud-Completion
Archive:  /content/drive/MyDrive/data/3Dfuture.zip
Base weight loaded 
Archive:  /content/drive/MyDrive/data/3Dfuture.zip
100
Base weight loaded 
0.17193955183029175 0.2027285099029541 0.0029690396040678024
test emd1: 0.213074 emd2: 0.171940 expansion_penalty: 0.002969 cd : 0.202729
[('base_model', 0.2027285099029541)]
m2
True
Previous weight loaded 
0.16358625888824463 0.16968634724617004 0.0028686411678791046
test emd1: 0.205849 emd2: 0.163586 expansion_penalty: 0.002869 cd : 0.169686
[('m2', 0.16968634724617004), ('base_model', 0.2027285099029541)]
m3
False
m4
True
Previous weight loaded 
0.1588815152645111 0.19322019815444946 0.0034917977172881365
test emd1: 0.225787 emd2: 0.158882 expansion_penalty: 0.003492 cd : 0.193220
[('m2', 0.16968634724617004), ('m4', 0.19322019815444946), ('base_model', 0.2027285099029541)]


```[('base_model', 0.0017337317415513098),
('m7', 0.0020423539844341577),
('m1', 0.003960765758529305),
('m10', 0.008972912528552115),
('m8', 0.011971391048282385)]
[('m16', 0.0020216004084795715), ('m15', 0.006628909669816494)] 'm17', 0.0023083634383510797
[('m1', 0.0015856193285435439)]

[('m2', 0.001651345631107688), ('m1', 0.0017085947841405868), ('base_model', 0.0017209897947032004)]

test emd1: 0.021308 emd2: 0.019169 expansion_penalty: 0.002754 cd : 0.000565
test emd1: 0.027037 emd2: 0.023807 expansion_penalty: 0.002644 cd : 0.000707
```